In [2]:
import numpy as np

In [10]:
from keras.models import Sequential
from keras.layers import Dense , Activation, Flatten, Embedding
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


# Create Embedding Index 

In [3]:
embeddings_index = {}

In [4]:
glove_data = '../practice/glove.6B/glove.6B.50d.txt'

In [7]:
f = open(glove_data,'rb')

In [8]:
for line in f:
    vector = line.split()
    word = vector[0]
    values = np.asarray(vector[1:],dtype='float32')
    embeddings_index[word] = values
f.close()
print("loaded %s word vectors",len(embeddings_index))

loaded %s word vectors 401413


In [9]:
text = ["The sun is shining in june!", "I like disney cartoons.", "Lelouch of the rebellion is in Code Geass"]

In [11]:
tokenizer = Tokenizer()

In [12]:
tokenizer.fit_on_texts(text)

In [14]:
X = tokenizer.texts_to_matrix(text)

In [46]:
y = [0,1,0] # predict whether a sentence contains like word or not

# Creating the embedding matrix

In [22]:
embedding_dimension = 10
word_index = tokenizer.word_index

In [27]:
embedding_matrix =     np.zeros((len(word_index)+1,embedding_dimension))

In [28]:
embedding_matrix

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [48]:
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector[:embedding_dimension]
        
    

In [49]:
embedding_matrix

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  4.18000013e-01,   2.49679998e-01,  -4.12420005e-01,
          1.21699996e-01,   3.45270008e-01,  -4.44569997e-02,
         -4.96879995e-01,  -1.78619996e-01,  -6.60229998e-04,
         -6.56599998e-01],
       [  6.18499994e-01,   6.42539978e-01,  -4.65519994e-01,
          3.75699997e-01,   7.48380005e-01,   5.37389994e-01,
          2.22390005e-03,  -6.05769992e-01,   2.64079988e-01,
          1.17030002e-01],
       [  3.30419987e-01,   2.49950007e-01,  -6.08739972e-01,
          1.09229997e-01,   3.63719985e-02,   1.50999993e-01,
         -5.50830007e-01,  -7.42390007e-02,  -9.23070014e-02,
         -3.28209996e-01],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00

In [51]:
embedding_matrix.shape # (number of words, embedding dimension)

(16, 10)

# Create embedding layer 

In [64]:
embedding_layer = Embedding(input_dim=embedding_matrix.shape[0],output_dim = embedding_matrix.shape[1], weights = [embedding_matrix]
                           , input_length= 16)

# Using pad sequences to change the input length size

In [54]:
from keras.preprocessing.sequence import pad_sequences

In [57]:
X.shape

(3, 16)

In [61]:
X

array([[ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,
         1.,  1.,  1.]])

In [59]:
pad_sequences(X,maxlen=12).shape

(3, 12)

In [63]:
pad_sequences(X,maxlen=12)

array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]])

# create model

In [71]:
model = Sequential()
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))
model.layers[0].trainable = False

In [74]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop')
model.fit(X, y=y, batch_size=20, epochs=700, verbose=0, validation_split=0.2,shuffle=True)
 

In [75]:
np.round(model.predict(X))

array([[ 0.],
       [ 1.],
       [ 0.]], dtype=float32)